In [1]:
!pip install transformers datasets evaluate accelerate -q

import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate

# =========================================================
# 1. PARAMETRELER
# =========================================================
MODEL_NAME = "bert-base-uncased"   # gerçek model
OUTPUT_DIR = "bert_imdb_final"     # çıkış klasörü
BATCH_SIZE = 16
EPOCHS = 1  # Hızlı test için 1 epoch
LEARNING_RATE = 2e-5

# GPU kontrolü
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️  Kullanılan cihaz: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Bellek: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

# =========================================================
# 2. GERÇEK HG FACE IMDB VERİ SETİ (FULL SET)
# =========================================================
print("\nGerçek IMDb veri seti indiriliyor...")
dataset = load_dataset("imdb")   # TAM veri seti (50k train + 25k test)
print(dataset)

# Eğitim + Test -> HuggingFace dataset formatı
# dataset["train"], dataset["test"]

# =========================================================
# 3. TOKENIZER & MODEL
# =========================================================
print("\nTokenizer ve model yükleniyor...")

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    token=False  # API TOKEN ENGELİ
)

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2,
    token=False
)

# Modeli GPU'ya taşı
model = model.to(device)
print(f"✅ Model {device} üzerine yüklendi")

# Tokenizasyon fonksiyonu
def tokenize_fn(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=256
    )

tokenized_dataset = dataset.map(tokenize_fn, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["text"])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch")

# =========================================================
# 4. METRİK (Accuracy)
# =========================================================
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# =========================================================
# 5. TRAINING ARGUMENTS
# =========================================================
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",          # evaluation_strategy -> eval_strategy
    save_strategy="epoch",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    load_best_model_at_end=True,
    push_to_hub=False,        # API kullanımını kapatır
    hub_token=None,
    hub_model_id=None,
    logging_steps=50,
    fp16=torch.cuda.is_available(),  # GPU varsa mixed precision kullan
    no_cuda=False,            # GPU kullanımını aktif et
)

# =========================================================
# 6. TRAINER
# =========================================================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# =========================================================
# 7. EĞİTİM
# =========================================================
print("\n📢 Eğitim başlıyor...\n")
trainer.train()

# =========================================================
# 8. TEST SETİ SONUÇLARI
# =========================================================
print("\n📌 Test sonuçları:")
results = trainer.evaluate()
print(results)

# =========================================================
# 9. MODEL KAYDETME
# =========================================================
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print(f"\n✅ Model ve tokenizer '{OUTPUT_DIR}' klasörüne kaydedildi.")



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


/home/mehmetcelik/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-11-17 20:51:55.070623: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-17 20:51:55.070623: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


🖥️  Kullanılan cihaz: cuda
GPU: NVIDIA GeForce RTX 3080 Laptop GPU
GPU Bellek: 7.66 GB

Gerçek IMDb veri seti indiriliyor...
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

Tokenizer ve model yükleniyor...
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

Tokenizer ve model yükleniyor...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Model cuda üzerine yüklendi


/tmp/ipykernel_262271/611884700.py:103: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



📢 Eğitim başlıyor...



Epoch,Training Loss,Validation Loss,Accuracy
1,0.221000,0.210170,0.919920



📌 Test sonuçları:


{'eval_loss': 0.21016967296600342, 'eval_accuracy': 0.91992, 'eval_runtime': 85.9573, 'eval_samples_per_second': 290.842, 'eval_steps_per_second': 18.183, 'epoch': 1.0}

✅ Model ve tokenizer 'bert_imdb_final' klasörüne kaydedildi.

✅ Model ve tokenizer 'bert_imdb_final' klasörüne kaydedildi.


In [2]:
# =========================================================
# 10. MODEL BAŞARI TESTİ - ÖRNEKLERİ TAHMİN ET
# =========================================================
print("\n" + "="*60)
print("📊 MODEL BAŞARI TESTİ - ÖRNEK TAHMİNLER")
print("="*60)

# Test için örnek yorumlar
test_examples = [
    "This movie was absolutely amazing! I loved every minute of it. The acting was superb!",
    "Terrible waste of time. The plot made no sense and the acting was horrible.",
    "One of the best films I've ever seen. Highly recommended!",
    "Boring and predictable. I fell asleep halfway through.",
    "A masterpiece of cinema. The director did an excellent job.",
    "I want my money back. This was the worst movie ever made."
]

labels_map = {0: "❌ NEGATİF", 1: "✅ POZİTİF"}

# Model tahminlerini yap
model.eval()
print("\n")
for idx, text in enumerate(test_examples, 1):
    # Tokenize
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Tahmin
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        prediction = torch.argmax(logits, dim=-1).item()
        probs = torch.softmax(logits, dim=-1)[0]
        confidence = probs[prediction].item() * 100
    
    print(f"Örnek {idx}:")
    print(f"  Yorum: {text}")
    print(f"  Tahmin: {labels_map[prediction]} (Güven: %{confidence:.1f})")
    print()

print("="*60)
print("✅ Test tamamlandı!")
print("="*60)



📊 MODEL BAŞARI TESTİ - ÖRNEK TAHMİNLER


Örnek 1:
  Yorum: This movie was absolutely amazing! I loved every minute of it. The acting was superb!
  Tahmin: ✅ POZİTİF (Güven: %99.5)

Örnek 2:
  Yorum: Terrible waste of time. The plot made no sense and the acting was horrible.
  Tahmin: ❌ NEGATİF (Güven: %99.7)

Örnek 3:
  Yorum: One of the best films I've ever seen. Highly recommended!
  Tahmin: ✅ POZİTİF (Güven: %99.4)

Örnek 4:
  Yorum: Boring and predictable. I fell asleep halfway through.
  Tahmin: ❌ NEGATİF (Güven: %97.5)

Örnek 5:
  Yorum: A masterpiece of cinema. The director did an excellent job.
  Tahmin: ✅ POZİTİF (Güven: %99.3)

Örnek 6:
  Yorum: I want my money back. This was the worst movie ever made.
  Tahmin: ❌ NEGATİF (Güven: %99.8)

✅ Test tamamlandı!
